# Ensuciar el dataframe original

In [74]:
import pandas as pd
import numpy as np
import random

def ensuciar_df(df_original):
    df = df_original.copy()  # Trabajar sobre una copia del DataFrame original

    # Duplicar algunas filas aleatorias
    num_duplicados = random.randint(1, int(len(df) * 0.3))  # Hasta 30% de duplicados máximo
    duplicadas = df.sample(n=num_duplicados, replace=True)  # Permitir reemplazo
    df = pd.concat([df, duplicadas], ignore_index=True)

    # Vectorizar la inserción de 'invalid' en el 2% de las celdas sin sobrescribir NaNs
    num_invalid = int(len(df) * 0.02)
    # Obtenemos la lista de columnas y removemos 'Date'
    columnas_para_invalid = [col for col in df.columns if col != 'Date']
    # Seleccionamos aleatoriamente de la lista filtrada
    for col in random.sample(columnas_para_invalid, len(columnas_para_invalid) // 2):
        # Convertir la columna a object si no lo es (para evitar problemas de tipo)
        if df[col].dtype != 'object':
            df[col] = df[col].astype('object')
        valid_indices = df.index[df[col].notna()]  # Obtener índices válidos que no son NaN
        if len(valid_indices) > 0:  # Verificar que hay índices válidos
            invalid_indices = np.random.choice(valid_indices, size=min(num_invalid, len(valid_indices)), replace=False)  # Elegir solo índices que no son NaN
            df.loc[invalid_indices, col] = 'bbb'

    # Cambiar tipos de formatos de algunas columnas aleatoriamente
    # Creamos la lista de columnas a modificar sin 'Date'
    columnas_disponibles = [col for col in df.columns if col != 'Date']
    columnas_a_cambiar = random.sample(columnas_disponibles, random.randint(1, len(columnas_disponibles) // 2))
    for col in columnas_a_cambiar:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].astype(str)  # Convertir a texto
        else:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Convertir a numérico donde sea posible

    # Vectorizar la inserción de NaNs en el 3% de las celdas
    num_nan = int(len(df) * 0.05)
    # Usamos la misma lista que en el paso anterior o creamos una nueva
    columnas_para_nan = [col for col in df.columns if col != 'Date']
    for column in columnas_para_nan:
        nan_indices = np.random.choice(df.index, size=min(num_nan, len(df)), replace=False)  # Elegir índices aleatorios
        df.loc[nan_indices, column] = np.nan  # Insertar NaNs

    return df

# Cargar un CSV y aplicar las modificaciones
ruta_csv = 'deutsche_bank.csv'  # Reemplaza con la ruta de tu archivo
df = pd.read_csv(ruta_csv)

max_intentos = 100
intento = 0
df_sucio = ensuciar_df(df)

# Columnas a excluir del conteo de NaNs
columnas_a_excluir = ['Date']

# Verificar la cantidad de NaNs y volver a ejecutar si es necesario
while intento < max_intentos:
    # Calcular el porcentaje de NaNs en cada columna (excluyendo las especificadas)
    nan_porcentaje = df_sucio.drop(columns=columnas_a_excluir).isnull().mean() * 100
    
    # Verificar si alguna columna tiene más del 10% de NaNs
    if (nan_porcentaje > 10).any():
        intento += 1
        print(f"Intento {intento}: Se encontraron columnas con más del 10% de NaNs. Volviendo a ensuciar el DataFrame.")
        df_sucio = ensuciar_df(df)
    else:
        print(f"Finalización en el intento {intento}. No se encontraron columnas con más del 10% de NaNs.")
        break

# Guardar el DataFrame sucio a un nuevo archivo CSV
df_sucio.to_csv('df_sucio.csv', index=False)

# Verificar la cantidad de NaNs en el DataFrame final
print(df_sucio.isnull().sum())

Finalización en el intento 0. No se encontraron columnas con más del 10% de NaNs.
Date                  0
Operating_Income    486
Expenses            351
Assets              351
Liabilities         351
Equity              351
Revenue             351
Cash_Flow           351
Net_Income          351
Debt_to_Equity      484
ROA                 351
Profit_Margin       351
Interest_Expense    351
Tax_Expense         482
Dividend_Payout     351
dtype: int64


In [75]:
df_sucio.isnull().sum()

Date                  0
Operating_Income    486
Expenses            351
Assets              351
Liabilities         351
Equity              351
Revenue             351
Cash_Flow           351
Net_Income          351
Debt_to_Equity      484
ROA                 351
Profit_Margin       351
Interest_Expense    351
Tax_Expense         482
Dividend_Payout     351
dtype: int64

In [76]:
df2=pd.read_csv("deutsche_bank_financial_performance.csv")
df2

,Date,Operating_Income,Expenses,Assets,Liabilities,Equity,Revenue,Cash_Flow,Net_Income,Debt_to_Equity,ROA,Profit_Margin,Interest_Expense,Tax_Expense,Dividend_Payout
0,1/1/2015,4370861.07,3682573.85,1.363403e+08,1.494296e+08,59803407.08,9435946.42,1428845.20,688287.22,2.50,0.01,0.07,609472.14,1042103.16,1170151.42
1,1/2/2015,9556428.76,1186425.69,1.955172e+08,4.725052e+07,55281922.53,12470620.28,1029017.28,8370003.06,0.85,0.04,0.67,699770.11,1329539.73,492998.93
2,1/3/2015,7587945.48,3093297.62,1.519954e+08,1.756590e+08,27447676.11,11882092.09,7320721.28,4494647.86,6.40,0.03,0.38,337977.34,345091.11,1603358.92
3,1/4/2015,6387926.36,3230217.71,2.097483e+08,1.382626e+08,87293509.41,4000698.76,1925965.75,3157708.65,1.58,0.02,0.79,1345468.95,1316373.62,426566.77
4,1/5/2015,2404167.76,2408588.02,8.124073e+07,2.773058e+08,70924670.47,3940243.11,6659376.16,-4420.26,3.91,0.00,0.00,175614.54,136655.30,2808563.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,3/6/2017,8849115.18,3102741.13,2.056449e+08,2.121410e+08,49789633.12,6165216.72,4084005.16,5746374.05,4.26,0.03,0.93,627993.14,835552.46,892162.90
796,3/7/2017,9389064.54,1733723.00,1.195069e+08,3.776259e+08,40096106.11,4135728.96,3164629.81,7655341.54,9.42,0.06,1.85,1046559.67,240065.80,707605.75
797,3/8/2017,6086198.65,857387.16,6.145149e+07,3.530014e+08,45511508.38,12181098.60,5618837.98,5228811.49,7.76,0.09,0.43,726107.79,1160927.80,1420453.37
798,3/9/2017,7269857.41,885462.12,3.406201e+08,5.897583e+07,57694652.82,3592472.21,5466741.19,6384395.29,1.02,0.02,1.78,937145.92,1402043.77,2734468.89


In [77]:
df2.isnull().sum()

Date                0
Operating_Income    0
Expenses            0
Assets              0
Liabilities         0
Equity              0
Revenue             0
Cash_Flow           0
Net_Income          0
Debt_to_Equity      0
ROA                 0
Profit_Margin       0
Interest_Expense    0
Tax_Expense         0
Dividend_Payout     0
dtype: int64